In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy zip file to local Colab storage
!cp "/content/drive/My Drive/cell_images.zip" "/content/"

# Extract in local storage (much faster)
!unzip -q "/content/cell_images.zip" -d "/content/extracted_data/"



Mounted at /content/drive


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random


In [3]:
data_path = 'extracted_data/cell_images'  # should contain 'Parasitized/' and 'Uninfected/'

print("Classes:", os.listdir(data_path))
print("Parasitized samples:", len(os.listdir(os.path.join(data_path, 'Parasitized'))))
print("Uninfected samples:", len(os.listdir(os.path.join(data_path, 'Uninfected'))))


Classes: ['cell_images', 'Parasitized', 'Uninfected']
Parasitized samples: 13780
Uninfected samples: 13780
